# 🕸️ Lección 6: Construcción de Spiders Avanzados

## 🎯 Objetivos

- Dominar diferentes tipos de spiders
- Implementar seguimiento de enlaces automático
- Manejar paginación compleja
- Optimizar el procesamiento paralelo
- Crear middlewares personalizados
- Manejar sitios web dinámicos

In [ ]:
import scrapy
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
from scrapy import signals
import json
import time
from datetime import datetime

print("🕸️ SPIDERS AVANZADOS")
print("=" * 25)
print("✅ Módulos importados correctamente")

## 🕷️ CrawlSpider: Seguimiento Automático de Enlaces

In [ ]:
class NewsCrawlSpider(CrawlSpider):
    """Spider que rastrea automáticamente enlaces de noticias"""
    name = 'news_crawler'
    allowed_domains = ['example-news.com']
    start_urls = ['http://example-news.com/']
    
    # Reglas para seguir enlaces
    rules = (
        # Seguir enlaces de categorías
        Rule(
            LinkExtractor(
                allow=(r'/category/',),
                deny=(r'/admin/', r'/login/')
            ),
            callback='parse_category',
            follow=True
        ),
        
        # Seguir enlaces de artículos
        Rule(
            LinkExtractor(
                allow=(r'/article/\d+/',),
                restrict_css='.article-links'
            ),
            callback='parse_article',
            follow=False
        ),
        
        # Seguir paginación
        Rule(
            LinkExtractor(
                allow=(r'\?page=\d+',),
                restrict_css='.pagination'
            ),
            follow=True
        )
    )
    
    def parse_category(self, response):
        """Procesar páginas de categorías"""
        category_name = response.css('h1::text').get()
        articles_count = len(response.css('.article-item'))
        
        yield {
            'type': 'category',
            'name': category_name,
            'articles_count': articles_count,
            'url': response.url
        }
    
    def parse_article(self, response):
        """Procesar artículos individuales"""
        yield {
            'type': 'article',
            'title': response.css('h1::text').get(),
            'content': response.css('.article-content::text').getall(),
            'author': response.css('.author::text').get(),
            'date': response.css('.date::text').get(),
            'category': response.css('.breadcrumb li:last-child::text').get(),
            'url': response.url,
            'word_count': len(' '.join(response.css('.article-content::text').getall()).split())
        }

print("🕷️ CRAWLSPIDER CREADO")
print("Características del CrawlSpider:")
print("  🔗 Seguimiento automático de enlaces")
print("  📋 Reglas configurables")
print("  🚫 Filtros allow/deny")
print("  📄 Paginación automática")
print("  🎯 Callbacks específicos")

## 🔄 Spider con Paginación Compleja

In [ ]:
class PaginationSpider(scrapy.Spider):
    """Spider que maneja diferentes tipos de paginación"""
    name = 'pagination_spider'
    
    def __init__(self, start_page=1, max_pages=10, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.start_page = int(start_page)
        self.max_pages = int(max_pages)
        self.current_page = self.start_page
        
        # URLs dinámicas
        self.base_url = 'http://example.com/products'
        self.start_urls = [f'{self.base_url}?page={self.start_page}']
    
    def parse(self, response):
        """Método principal con múltiples estrategias de paginación"""
        
        # Extraer productos de la página actual
        products = response.css('.product-item')
        
        for product in products:
            yield {
                'name': product.css('.product-name::text').get(),
                'price': product.css('.price::text').get(),
                'rating': product.css('.rating::attr(data-rating)').get(),
                'page': self.current_page,
                'url': response.url
            }
        
        # Estrategia 1: Paginación con enlaces "Next"
        next_page = response.css('.pagination .next::attr(href)').get()
        if next_page and self.current_page < self.max_pages:
            self.current_page += 1
            yield response.follow(next_page, self.parse)
        
        # Estrategia 2: Paginación con números de página
        elif not next_page and self.current_page < self.max_pages:
            next_page_num = self.current_page + 1
            next_url = f'{self.base_url}?page={next_page_num}'
            
            # Verificar si existe la próxima página
            yield scrapy.Request(
                next_url,
                callback=self.parse,
                errback=self.handle_pagination_error
            )
            self.current_page += 1
        
        # Estrategia 3: Paginación AJAX (simulada)
        if response.css('.load-more-button'):
            yield self.make_ajax_request(response)
    
    def make_ajax_request(self, response):
        """Simular request AJAX para más contenido"""
        ajax_url = response.urljoin('/api/products')
        
        return scrapy.Request(
            ajax_url,
            method='POST',
            headers={'Content-Type': 'application/json'},
            body=json.dumps({
                'page': self.current_page + 1,
                'limit': 20
            }),
            callback=self.parse_ajax_response
        )
    
    def parse_ajax_response(self, response):
        """Procesar respuesta AJAX JSON"""
        data = json.loads(response.text)
        
        for item in data.get('products', []):
            yield {
                'name': item.get('name'),
                'price': item.get('price'),
                'rating': item.get('rating'),
                'page': self.current_page,
                'source': 'ajax'
            }
        
        # Continuar si hay más páginas
        if data.get('has_next') and self.current_page < self.max_pages:
            self.current_page += 1
            yield self.make_ajax_request(response)
    
    def handle_pagination_error(self, failure):
        """Manejar errores en paginación"""
        self.logger.error(f'Error en paginación: {failure.value}')
        # Detener paginación en caso de error 404
        if hasattr(failure.value, 'response') and failure.value.response.status == 404:
            self.logger.info('Fin de páginas alcanzado (404)')

print("🔄 SPIDER DE PAGINACIÓN CREADO")
print("Estrategias de paginación:")
print("  1. 🔗 Enlaces 'Next' tradicionales")
print("  2. 🔢 Números de página secuenciales")
print("  3. ⚡ Carga dinámica AJAX")
print("  4. ❌ Manejo de errores y límites")

## 🚀 Spider con Procesamiento Paralelo

In [ ]:
class ParallelSpider(scrapy.Spider):
    """Spider optimizado para procesamiento paralelo"""
    name = 'parallel_spider'
    
    custom_settings = {
        'CONCURRENT_REQUESTS': 32,  # Aumentar requests concurrentes
        'CONCURRENT_REQUESTS_PER_DOMAIN': 16,
        'DOWNLOAD_DELAY': 0.25,  # Delay mínimo
        'RANDOMIZE_DOWNLOAD_DELAY': 0.5,
        'AUTOTHROTTLE_ENABLED': True,  # Auto-ajuste de velocidad
        'AUTOTHROTTLE_START_DELAY': 0.25,
        'AUTOTHROTTLE_MAX_DELAY': 10,
        'AUTOTHROTTLE_TARGET_CONCURRENCY': 2.0,
        'AUTOTHROTTLE_DEBUG': True,
    }
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.stats = {
            'requests_made': 0,
            'responses_received': 0,
            'items_scraped': 0,
            'errors': 0,
            'start_time': time.time()
        }
    
    def start_requests(self):
        """Generar múltiples requests iniciales para paralelización"""
        # Generar URLs para múltiples categorías simultáneamente
        categories = ['electronics', 'books', 'clothing', 'home', 'sports']
        
        for category in categories:
            for page in range(1, 6):  # 5 páginas por categoría
                url = f'http://example.com/{category}?page={page}'
                self.stats['requests_made'] += 1
                
                yield scrapy.Request(
                    url,
                    callback=self.parse_category,
                    meta={
                        'category': category,
                        'page': page,
                        'priority': 10 - page  # Prioridad mayor para primeras páginas
                    }
                )
    
    def parse_category(self, response):
        """Procesar páginas de categoría en paralelo"""
        self.stats['responses_received'] += 1
        category = response.meta['category']
        page = response.meta['page']
        
        # Extraer productos
        products = response.css('.product-item')
        
        for product in products:
            product_url = product.css('a::attr(href)').get()
            if product_url:
                # Crear request para página de producto con alta prioridad
                yield scrapy.Request(
                    response.urljoin(product_url),
                    callback=self.parse_product,
                    meta={
                        'category': category,
                        'source_page': page
                    },
                    priority=15  # Alta prioridad para productos
                )
    
    def parse_product(self, response):
        """Procesar productos individuales"""
        try:
            # Extracción rápida y eficiente
            item = {
                'name': response.css('h1::text').get(),
                'price': self.extract_price(response),
                'category': response.meta['category'],
                'description': self.extract_description(response),
                'rating': response.css('.rating::attr(data-rating)').get(),
                'availability': response.css('.availability::text').get(),
                'sku': response.css('[data-sku]::attr(data-sku)').get(),
                'url': response.url,
                'scraped_at': datetime.now().isoformat()
            }
            
            self.stats['items_scraped'] += 1
            
            # Log progreso cada 100 items
            if self.stats['items_scraped'] % 100 == 0:
                elapsed = time.time() - self.stats['start_time']
                rate = self.stats['items_scraped'] / elapsed
                self.logger.info(f'Scraped {self.stats["items_scraped"]} items at {rate:.2f} items/sec')
            
            yield item
            
        except Exception as e:
            self.stats['errors'] += 1
            self.logger.error(f'Error parsing product {response.url}: {e}')
    
    def extract_price(self, response):
        """Extraer precio con múltiples selectores"""
        price_selectors = [
            '.price::text',
            '.current-price::text',
            '[data-price]::attr(data-price)',
            '.price-now::text'
        ]
        
        for selector in price_selectors:
            price = response.css(selector).get()
            if price:
                return price.strip()
        return None
    
    def extract_description(self, response):
        """Extraer descripción limitando longitud"""
        description = response.css('.description::text').getall()
        if description:
            full_desc = ' '.join(description).strip()
            return full_desc[:500] + '...' if len(full_desc) > 500 else full_desc
        return None
    
    def closed(self, reason):
        """Estadísticas finales cuando termina el spider"""
        elapsed = time.time() - self.stats['start_time']
        
        self.logger.info('=== ESTADÍSTICAS FINALES ===')
        self.logger.info(f'Tiempo total: {elapsed:.2f} segundos')
        self.logger.info(f'Requests realizados: {self.stats["requests_made"]}')
        self.logger.info(f'Responses recibidas: {self.stats["responses_received"]}')
        self.logger.info(f'Items scrapeados: {self.stats["items_scraped"]}')
        self.logger.info(f'Errores: {self.stats["errors"]}')
        
        if elapsed > 0:
            self.logger.info(f'Velocidad promedio: {self.stats["items_scraped"] / elapsed:.2f} items/sec')

print("🚀 SPIDER PARALELO CREADO")
print("Optimizaciones implementadas:")
print("  ⚡ 32 requests concurrentes")
print("  🎯 Sistema de prioridades")
print("  📊 Auto-throttling inteligente")
print("  📈 Métricas en tiempo real")
print("  🔧 Extracción optimizada")

## 🛡️ Middleware Avanzado para Anti-Detección

In [ ]:
import random
from scrapy.downloadermiddlewares.useragent import UserAgentMiddleware
from scrapy import signals

class AntiDetectionMiddleware:
    """Middleware avanzado para evitar detección"""
    
    def __init__(self, crawler):
        self.crawler = crawler
        
        # Pool de User Agents realistas
        self.user_agents = [
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/121.0',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2 Safari/605.1.15',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        ]
        
        # Headers adicionales para parecer más humano
        self.common_headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'DNT': '1',
            'Connection': 'keep-alive',
        }
        
        # Estadísticas de requests
        self.requests_count = 0
        self.blocks_detected = 0
    
    @classmethod
    def from_crawler(cls, crawler):
        return cls(crawler)
    
    def process_request(self, request, spider):
        """Procesar cada request para evitar detección"""
        self.requests_count += 1
        
        # Rotar User Agent
        request.headers['User-Agent'] = random.choice(self.user_agents)
        
        # Agregar headers comunes
        for header, value in self.common_headers.items():
            request.headers[header] = value
        
        # Agregar headers específicos del sitio
        if 'amazon' in request.url:
            request.headers['Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'
            request.headers['Upgrade-Insecure-Requests'] = '1'
        
        # Simular comportamiento humano con delays variables
        if hasattr(request, 'meta'):
            if not request.meta.get('dont_delay'):
                request.meta['download_delay'] = random.uniform(1, 3)
        
        # Log cada 100 requests
        if self.requests_count % 100 == 0:
            spider.logger.info(f'Processed {self.requests_count} requests, {self.blocks_detected} blocks detected')
        
        return None
    
    def process_response(self, request, response, spider):
        """Analizar respuestas para detectar bloqueos"""
        
        # Detectar páginas de bloqueo comunes
        block_indicators = [
            'access denied',
            'blocked',
            'captcha',
            'robot',
            'suspicious activity',
            'rate limit'
        ]
        
        response_text = response.text.lower()
        
        for indicator in block_indicators:
            if indicator in response_text:
                self.blocks_detected += 1
                spider.logger.warning(f'Possible block detected on {request.url}: {indicator}')
                
                # Crear nuevo request con delay mayor
                new_request = request.copy()
                new_request.meta['download_delay'] = 30  # Esperar 30 segundos
                new_request.meta['retry_count'] = request.meta.get('retry_count', 0) + 1
                
                if new_request.meta['retry_count'] < 3:
                    return new_request
                else:
                    spider.logger.error(f'Max retries reached for {request.url}')
        
        return response
    
    def process_exception(self, request, exception, spider):
        """Manejar excepciones de red"""
        spider.logger.error(f'Exception for {request.url}: {exception}')
        return None

# Middleware para gestión inteligente de cookies
class SmartCookieMiddleware:
    """Middleware para manejo inteligente de cookies"""
    
    def __init__(self):
        self.session_cookies = {}
    
    def process_request(self, request, spider):
        """Agregar cookies persistentes"""
        domain = self.get_domain(request.url)
        
        if domain in self.session_cookies:
            request.cookies.update(self.session_cookies[domain])
        
        return None
    
    def process_response(self, request, response, spider):
        """Guardar cookies de sesión"""
        if response.headers.getlist('Set-Cookie'):
            domain = self.get_domain(request.url)
            
            if domain not in self.session_cookies:
                self.session_cookies[domain] = {}
            
            # Extraer y guardar cookies importantes
            for cookie_header in response.headers.getlist('Set-Cookie'):
                cookie_str = cookie_header.decode('utf-8')
                if 'session' in cookie_str.lower() or 'auth' in cookie_str.lower():
                    # Parsear y guardar cookie importante
                    cookie_parts = cookie_str.split(';')[0].split('=')
                    if len(cookie_parts) == 2:
                        self.session_cookies[domain][cookie_parts[0]] = cookie_parts[1]
        
        return response
    
    def get_domain(self, url):
        """Extraer dominio de URL"""
        from urllib.parse import urlparse
        return urlparse(url).netloc

print("🛡️ MIDDLEWARES ANTI-DETECCIÓN CREADOS")
print("Características de seguridad:")
print("  🎭 Rotación de User Agents")
print("  🔍 Detección de bloqueos")
print("  🍪 Gestión inteligente de cookies")
print("  ⏱️ Delays variables")
print("  🔄 Sistema de reintentos")

## 🖥️ Spider para Sitios con JavaScript (Selenium Integration)

In [ ]:
# Simulación de integración con Selenium
class JavaScriptSpider(scrapy.Spider):
    """Spider que maneja contenido JavaScript con Selenium"""
    name = 'javascript_spider'
    
    custom_settings = {
        'DOWNLOAD_DELAY': 3,  # Mayor delay para JS
        'DOWNLOADER_MIDDLEWARES': {
            'scrapy_selenium.SeleniumMiddleware': 800
        }
    }
    
    def start_requests(self):
        """Requests iniciales con configuración de Selenium"""
        urls = [
            'https://spa-example.com/products',
            'https://infinite-scroll.com/items'
        ]
        
        for url in urls:
            yield scrapy.Request(
                url,
                callback=self.parse_spa,
                meta={
                    'selenium': True,
                    'selenium_wait_time': 10,
                    'selenium_wait_condition': 'presence_of_element_located',
                    'selenium_wait_condition_arg': '.product-item'
                }
            )
    
    def parse_spa(self, response):
        """Procesar Single Page Application"""
        # En este punto, Selenium ya cargó el JavaScript
        products = response.css('.product-item')
        
        for product in products:
            # Extraer datos que fueron generados por JavaScript
            yield {
                'name': product.css('.js-product-name::text').get(),
                'price': product.css('[data-price]::attr(data-price)').get(),
                'dynamic_rating': product.css('.js-rating::text').get(),
                'availability': product.css('.js-stock::text').get(),
                'url': response.url
            }
        
        # Manejar infinite scroll
        if response.css('.load-more-button'):
            yield scrapy.Request(
                response.url,
                callback=self.handle_infinite_scroll,
                meta={
                    'selenium': True,
                    'selenium_action': 'click_and_wait',
                    'selenium_action_target': '.load-more-button',
                    'selenium_wait_time': 5
                }
            )
    
    def handle_infinite_scroll(self, response):
        """Manejar carga infinita de contenido"""
        # Procesar nuevos productos cargados
        new_products = response.css('.product-item')
        
        for product in new_products:
            yield {
                'name': product.css('.js-product-name::text').get(),
                'price': product.css('[data-price]::attr(data-price)').get(),
                'source': 'infinite_scroll',
                'url': response.url
            }
        
        # Continuar scroll si hay más contenido
        if response.css('.load-more-button:not(.disabled)'):
            yield scrapy.Request(
                response.url,
                callback=self.handle_infinite_scroll,
                meta=response.meta
            )

# Simulación de configuración para Selenium
SELENIUM_DRIVER_SETTINGS = {
    'SELENIUM_DRIVER_NAME': 'chrome',
    'SELENIUM_DRIVER_EXECUTABLE_PATH': None,  # Usar webdriver-manager
    'SELENIUM_DRIVER_ARGUMENTS': [
        '--headless',  # Ejecutar sin ventana
        '--no-sandbox',
        '--disable-dev-shm-usage',
        '--disable-gpu',
        '--window-size=1920,1080',
        '--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    ]
}

print("🖥️ SPIDER JAVASCRIPT CREADO")
print("Capacidades JavaScript:")
print("  ⚡ Integración con Selenium")
print("  🔄 Infinite scroll automático")
print("  ⏳ Esperas inteligentes")
print("  🖱️ Interacciones con botones")
print("  📱 SPA (Single Page Apps)")

print("\n⚙️ Configuración Selenium:")
for key, value in SELENIUM_DRIVER_SETTINGS.items():
    print(f"  {key}: {value}")

## 📊 Spider con Monitoreo Avanzado

In [ ]:
class MonitoredSpider(scrapy.Spider):
    """Spider con sistema de monitoreo y alertas"""
    name = 'monitored_spider'
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        # Métricas detalladas
        self.metrics = {
            'start_time': time.time(),
            'pages_scraped': 0,
            'items_extracted': 0,
            'errors_encountered': 0,
            'response_times': [],
            'status_codes': {},
            'domains_scraped': set(),
            'data_quality_issues': 0
        }
        
        # Umbrales para alertas
        self.thresholds = {
            'max_error_rate': 0.05,  # 5% de error máximo
            'min_items_per_minute': 10,
            'max_response_time': 30,  # segundos
            'max_consecutive_errors': 10
        }
        
        self.consecutive_errors = 0
        self.last_report_time = time.time()
    
    @classmethod
    def from_crawler(cls, crawler, *args, **kwargs):
        spider = super().from_crawler(crawler, *args, **kwargs)
        
        # Conectar signals
        crawler.signals.connect(spider.spider_opened, signal=signals.spider_opened)
        crawler.signals.connect(spider.spider_closed, signal=signals.spider_closed)
        crawler.signals.connect(spider.request_scheduled, signal=signals.request_scheduled)
        crawler.signals.connect(spider.response_received, signal=signals.response_received)
        crawler.signals.connect(spider.item_scraped, signal=signals.item_scraped)
        
        return spider
    
    def spider_opened(self, spider):
        """Inicializar monitoreo"""
        self.logger.info(f'🕷️ Spider {spider.name} iniciado con monitoreo')
        self.logger.info(f'📊 Umbrales configurados: {self.thresholds}')
    
    def request_scheduled(self, request, spider):
        """Monitorear requests programados"""
        request.meta['request_start_time'] = time.time()
        
        # Rastrear dominios
        from urllib.parse import urlparse
        domain = urlparse(request.url).netloc
        self.metrics['domains_scraped'].add(domain)
    
    def response_received(self, response, request, spider):
        """Analizar respuestas recibidas"""
        # Calcular tiempo de respuesta
        if 'request_start_time' in request.meta:
            response_time = time.time() - request.meta['request_start_time']
            self.metrics['response_times'].append(response_time)
            
            # Alertar si el tiempo de respuesta es muy alto
            if response_time > self.thresholds['max_response_time']:
                self.logger.warning(f'⚠️ Tiempo de respuesta alto: {response_time:.2f}s para {request.url}')
        
        # Rastrear códigos de estado
        status = response.status
        self.metrics['status_codes'][status] = self.metrics['status_codes'].get(status, 0) + 1
        
        # Detectar errores
        if status >= 400:
            self.metrics['errors_encountered'] += 1
            self.consecutive_errors += 1
            
            if self.consecutive_errors >= self.thresholds['max_consecutive_errors']:
                self.send_alert(f'🚨 {self.consecutive_errors} errores consecutivos detectados')
        else:
            self.consecutive_errors = 0
        
        self.metrics['pages_scraped'] += 1
        
        # Reporte periódico cada 5 minutos
        if time.time() - self.last_report_time > 300:  # 5 minutos
            self.generate_progress_report()
            self.last_report_time = time.time()
    
    def item_scraped(self, item, response, spider):
        """Monitorear items extraídos"""
        self.metrics['items_extracted'] += 1
        
        # Verificar calidad de datos
        if self.check_data_quality(item):
            self.metrics['data_quality_issues'] += 1
    
    def check_data_quality(self, item):
        """Verificar calidad de los datos extraídos"""
        issues = 0
        
        # Verificar campos vacíos
        for key, value in item.items():
            if not value or (isinstance(value, str) and len(value.strip()) == 0):
                issues += 1
        
        # Verificar datos sospechosos
        if 'price' in item:
            price_str = str(item['price'])
            if 'error' in price_str.lower() or 'null' in price_str.lower():
                issues += 1
        
        return issues > 0
    
    def generate_progress_report(self):
        """Generar reporte de progreso"""
        elapsed = time.time() - self.metrics['start_time']
        
        # Calcular métricas
        pages_per_minute = (self.metrics['pages_scraped'] / elapsed) * 60
        items_per_minute = (self.metrics['items_extracted'] / elapsed) * 60
        error_rate = self.metrics['errors_encountered'] / max(self.metrics['pages_scraped'], 1)
        avg_response_time = sum(self.metrics['response_times']) / max(len(self.metrics['response_times']), 1)
        
        # Log reporte
        self.logger.info('📊 === REPORTE DE PROGRESO ===')
        self.logger.info(f'⏱️ Tiempo transcurrido: {elapsed/60:.1f} minutos')
        self.logger.info(f'📄 Páginas scrapeadas: {self.metrics["pages_scraped"]}')
        self.logger.info(f'📦 Items extraídos: {self.metrics["items_extracted"]}')
        self.logger.info(f'🌐 Dominios: {len(self.metrics["domains_scraped"])}')
        self.logger.info(f'⚡ Velocidad: {pages_per_minute:.1f} páginas/min, {items_per_minute:.1f} items/min')
        self.logger.info(f'❌ Tasa de error: {error_rate:.3f} ({self.metrics["errors_encountered"]} errores)')
        self.logger.info(f'🕐 Tiempo promedio de respuesta: {avg_response_time:.2f}s')
        self.logger.info(f'⚠️ Problemas de calidad: {self.metrics["data_quality_issues"]}')
        
        # Verificar umbrales y enviar alertas
        if error_rate > self.thresholds['max_error_rate']:
            self.send_alert(f'🚨 Tasa de error alta: {error_rate:.3f}')
        
        if items_per_minute < self.thresholds['min_items_per_minute']:
            self.send_alert(f'🐌 Velocidad baja: {items_per_minute:.1f} items/min')
    
    def send_alert(self, message):
        """Enviar alerta (simulado)"""
        self.logger.error(f'ALERTA: {message}')
        # Aquí se integraría con Slack, email, webhook, etc.
    
    def spider_closed(self, spider):
        """Generar reporte final"""
        elapsed = time.time() - self.metrics['start_time']
        
        self.logger.info('🏁 === REPORTE FINAL ===')
        self.logger.info(f'⏱️ Duración total: {elapsed/60:.1f} minutos')
        self.logger.info(f'📊 Métricas finales: {self.metrics["pages_scraped"]} páginas, {self.metrics["items_extracted"]} items')
        self.logger.info(f'🌐 Dominios scrapeados: {", ".join(self.metrics["domains_scraped"])}')
        self.logger.info(f'📈 Códigos de estado: {self.metrics["status_codes"]}')
        
        if self.metrics['response_times']:
            avg_rt = sum(self.metrics['response_times']) / len(self.metrics['response_times'])
            max_rt = max(self.metrics['response_times'])
            self.logger.info(f'🕐 Tiempo de respuesta: promedio {avg_rt:.2f}s, máximo {max_rt:.2f}s')

print("📊 SPIDER MONITOREADO CREADO")
print("Sistema de monitoreo:")
print("  📈 Métricas en tiempo real")
print("  🚨 Alertas automáticas")
print("  📋 Reportes periódicos")
print("  🔍 Control de calidad de datos")
print("  ⏱️ Análisis de rendimiento")

## 🎯 Ejercicio Práctico Completo

In [ ]:
print("🎯 EJERCICIO PRÁCTICO: SPIDER MASTER")
print("=" * 45)

class MasterSpider(scrapy.Spider):
    """Spider que combina todas las técnicas avanzadas"""
    name = 'master_spider'
    allowed_domains = ['example-store.com']
    
    custom_settings = {
        # Configuración de rendimiento
        'CONCURRENT_REQUESTS': 16,
        'CONCURRENT_REQUESTS_PER_DOMAIN': 8,
        'DOWNLOAD_DELAY': 1,
        'RANDOMIZE_DOWNLOAD_DELAY': 0.5,
        
        # Auto-throttling
        'AUTOTHROTTLE_ENABLED': True,
        'AUTOTHROTTLE_START_DELAY': 0.5,
        'AUTOTHROTTLE_MAX_DELAY': 10,
        'AUTOTHROTTLE_TARGET_CONCURRENCY': 2.0,
        
        # Middlewares
        'DOWNLOADER_MIDDLEWARES': {
            '__main__.AntiDetectionMiddleware': 500,
            '__main__.SmartCookieMiddleware': 600,
        },
        
        # Pipelines
        'ITEM_PIPELINES': {
            '__main__.ValidationPipeline': 300,
            '__main__.ProcessingPipeline': 400,
        },
        
        # Export
        'FEEDS': {
            'master_output.json': {
                'format': 'json',
                'encoding': 'utf8',
                'indent': 2
            },
            'master_output.csv': {
                'format': 'csv',
                'encoding': 'utf8'
            }
        },
        
        # Logging
        'LOG_LEVEL': 'INFO'
    }
    
    def __init__(self, category='all', max_pages=5, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.category = category
        self.max_pages = int(max_pages)
        self.scraped_urls = set()
        
        # Configurar URLs iniciales
        if category == 'all':
            self.start_urls = [
                'https://example-store.com/categories/electronics',
                'https://example-store.com/categories/books',
                'https://example-store.com/categories/clothing'
            ]
        else:
            self.start_urls = [f'https://example-store.com/categories/{category}']
    
    def parse(self, response):
        """Parse principal con múltiples estrategias"""
        # Extraer categoría actual
        category = self.extract_category(response)
        
        # Estrategia 1: Productos en página de categoría
        products = response.css('.product-item')
        
        for product in products:
            product_url = product.css('a::attr(href)').get()
            if product_url and product_url not in self.scraped_urls:
                self.scraped_urls.add(product_url)
                
                yield response.follow(
                    product_url,
                    callback=self.parse_product,
                    meta={'category': category}
                )
        
        # Estrategia 2: Paginación
        next_page = response.css('.pagination .next::attr(href)').get()
        current_page = self.get_current_page(response)
        
        if next_page and current_page < self.max_pages:
            yield response.follow(next_page, self.parse)
        
        # Estrategia 3: Subcategorías
        subcategories = response.css('.subcategory-link::attr(href)').getall()
        for subcat_url in subcategories[:3]:  # Limitar a 3 subcategorías
            yield response.follow(subcat_url, self.parse)
    
    def parse_product(self, response):
        """Extraer información detallada del producto"""
        try:
            # Datos básicos
            item = {
                'url': response.url,
                'name': response.css('h1::text').get(),
                'price': self.extract_price(response),
                'category': response.meta.get('category'),
                'brand': response.css('.brand::text').get(),
                'sku': response.css('[data-sku]::attr(data-sku)').get(),
                'availability': response.css('.availability::text').get(),
                'rating': self.extract_rating(response),
                'reviews_count': self.extract_reviews_count(response),
                'description': self.extract_description(response),
                'images': response.css('.product-images img::attr(src)').getall(),
                'specifications': self.extract_specifications(response),
                'scraped_at': datetime.now().isoformat()
            }
            
            # Validación básica
            if item['name'] and item['price']:
                yield item
            else:
                self.logger.warning(f'Producto incompleto en {response.url}')
                
        except Exception as e:
            self.logger.error(f'Error parsing product {response.url}: {e}')
    
    def extract_category(self, response):
        """Extraer categoría de la página"""
        breadcrumb = response.css('.breadcrumb li:last-child::text').get()
        if breadcrumb:
            return breadcrumb.strip()
        
        # Fallback: extraer de URL
        from urllib.parse import urlparse
        path_parts = urlparse(response.url).path.split('/')
        if 'categories' in path_parts:
            idx = path_parts.index('categories')
            if idx + 1 < len(path_parts):
                return path_parts[idx + 1]
        
        return 'unknown'
    
    def extract_price(self, response):
        """Extraer precio con múltiples selectores"""
        selectors = [
            '.price::text',
            '.current-price::text',
            '[data-price]::attr(data-price)',
            '.price-now::text'
        ]
        
        for selector in selectors:
            price = response.css(selector).get()
            if price:
                # Limpiar precio
                import re
                price_clean = re.sub(r'[^\d.,]', '', price)
                return price_clean
        
        return None
    
    def extract_rating(self, response):
        """Extraer rating del producto"""
        # Múltiples formatos de rating
        rating_selectors = [
            '.rating::attr(data-rating)',
            '.stars::attr(data-stars)',
            '.review-score::text'
        ]
        
        for selector in rating_selectors:
            rating = response.css(selector).get()
            if rating:
                try:
                    return float(rating)
                except ValueError:
                    continue
        
        # Contar estrellas visuales
        filled_stars = response.css('.star.filled, .star.active').getall()
        if filled_stars:
            return len(filled_stars)
        
        return None
    
    def extract_reviews_count(self, response):
        """Extraer número de reviews"""
        reviews_text = response.css('.reviews-count::text').get()
        if reviews_text:
            import re
            match = re.search(r'(\d+)', reviews_text)
            if match:
                return int(match.group(1))
        return 0
    
    def extract_description(self, response):
        """Extraer descripción del producto"""
        desc_selectors = [
            '.product-description::text',
            '.description p::text',
            '.product-details::text'
        ]
        
        for selector in desc_selectors:
            desc_parts = response.css(selector).getall()
            if desc_parts:
                full_desc = ' '.join(desc_parts).strip()
                return full_desc[:1000]  # Limitar longitud
        
        return None
    
    def extract_specifications(self, response):
        """Extraer especificaciones técnicas"""
        specs = {}
        
        # Tabla de especificaciones
        spec_rows = response.css('.specifications tr')
        for row in spec_rows:
            label = row.css('td:first-child::text').get()
            value = row.css('td:last-child::text').get()
            if label and value:
                specs[label.strip().lower().replace(' ', '_')] = value.strip()
        
        # Lista de características
        features = response.css('.features li::text').getall()
        if features:
            specs['features'] = features
        
        return specs
    
    def get_current_page(self, response):
        """Obtener número de página actual"""
        # Extraer de URL
        from urllib.parse import urlparse, parse_qs
        query_params = parse_qs(urlparse(response.url).query)
        page = query_params.get('page', ['1'])[0]
        
        try:
            return int(page)
        except ValueError:
            return 1

print("🏆 MASTER SPIDER CREADO")
print("Técnicas integradas:")
print("  🕷️ CrawlSpider avanzado")
print("  🔄 Paginación múltiple")
print("  🚀 Procesamiento paralelo")
print("  🛡️ Anti-detección")
print("  📊 Monitoreo completo")
print("  🎯 Extracción robusta")

print("\n🎯 COMANDOS DE EJECUCIÓN:")
print("  scrapy crawl master_spider")
print("  scrapy crawl master_spider -a category=electronics")
print("  scrapy crawl master_spider -a max_pages=10")
print("  scrapy crawl master_spider -s LOG_LEVEL=DEBUG")

## 📖 Resumen de la Lección

### 🎯 Técnicas Dominadas

1. **CrawlSpider Avanzado**:
   - Seguimiento automático de enlaces con reglas
   - LinkExtractor con filtros allow/deny
   - Callbacks específicos para diferentes tipos de páginas
   - Restricciones por CSS y XPath

2. **Paginación Compleja**:
   - Enlaces "Next" tradicionales
   - Paginación numérica secuencial
   - Infinite scroll con AJAX
   - Manejo de errores 404

3. **Procesamiento Paralelo**:
   - Configuración de concurrencia optimizada
   - Sistema de prioridades para requests
   - Auto-throttling inteligente
   - Métricas de rendimiento en tiempo real

4. **Middlewares Anti-Detección**:
   - Rotación de User Agents
   - Gestión inteligente de cookies
   - Detección automática de bloqueos
   - Delays variables y headers realistas

5. **Integración con JavaScript**:
   - Selenium middleware
   - Esperas condicionales
   - Interacciones con elementos dinámicos
   - Manejo de SPAs

6. **Monitoreo Avanzado**:
   - Métricas detalladas en tiempo real
   - Sistema de alertas automático
   - Control de calidad de datos
   - Reportes periódicos y finales

### 🚀 Próxima Lección: Procesamiento y Almacenamiento

En la siguiente lección aprenderemos:
- Limpieza y validación avanzada de datos
- Integración con bases de datos (SQL y NoSQL)
- Pipelines de transformación complejos
- Export a múltiples formatos
- Data warehousing y ETL

### 💡 Mejores Prácticas Aprendidas

1. **Diseño Escalable**: Usar configuraciones flexibles y parametrizables
2. **Monitoreo Proactivo**: Implementar alertas y métricas desde el inicio
3. **Robustez**: Manejar errores gracefully y tener fallbacks
4. **Eficiencia**: Optimizar concurrencia sin sobrecargar servidores
5. **Mantenibilidad**: Código modular y bien documentado

### 🛠️ Herramientas Master

```python
# Configuración de alto rendimiento
CONCURRENT_REQUESTS = 32
AUTOTHROTTLE_ENABLED = True
DOWNLOAD_DELAY = 0.25

# Middlewares esenciales
AntiDetectionMiddleware
SmartCookieMiddleware
MonitoringMiddleware

# Técnicas de extracción
multiple_selectors_fallback()
intelligent_data_validation()
robust_error_handling()
```

---

¡Increíble! 🎉 Ahora eres un experto en spiders avanzados y puedes manejar los sitios web más complejos con confianza.